In [1]:
from jqdata import *
import pandas as pd 
import numpy as np
import statsmodels.api as sm 


In [2]:
%%time
#########获取数据############
##########################################期货标的################################################
#######起始日期
start_date = '2015-02-09'
######终止日期
end_date = '2023-02-24'
######标的
# 上证50ETF期权， 510050.XSHG,上市时间为2015-02-09，从2015-05-01开始取数据
# 沪深300ETF期权（上海交易所），510300.XSHG，上市时间为2019-12-23
# 沪深300ETF期权（深圳交易所），159919，上市时间为2019-12-23
# 沪深300股指期货期权，IO，看涨期权：IO合约月份-C-行权价格，如IO2302-C-3350，看跌期权：IO合约月份-P-行权价格
# 中证1000股指期货期权，MO

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 13.1 µs


In [3]:
trade_days=get_trade_days(start_date, end_date)

In [4]:
trade_days

array([2015-02-09, 2015-02-10, 2015-02-11, ..., 2023-02-22, 2023-02-23,
       2023-02-24], dtype=object)

In [5]:
%%time
# t = -1
# date = trade_days[t]
# price=get_price('510050.XSHG',count=1,end_date=date,fq='none',fields=['open'])['open'].values[0]
def get_daily_contracts(date):
#     q=query(jy.Opt_DailyPreOpen).filter(jy.Opt_DailyPreOpen.TradingDate==date,jy.Opt_DailyPreOpen.ULAName=='50ETF')
    q = query(opt.OPT_DAILY_PREOPEN).filter(opt.OPT_DAILY_PREOPEN.underlying_symbol=='510050.XSHG',
                                            opt.OPT_DAILY_PREOPEN.date==date)
#     df=jy.run_query(q)
    df = opt.run_query(q)
    # print(df.head().T)
#     df=df.loc[:,['ContractCode','TradingCode','StrikePrice','ExerciseDate']]
    df = df[['code','trading_code','exercise_price','exercise_date']]
#     df=df[df['ExerciseDate']==df['ExerciseDate'].min()].reset_index()#这个功能是干什么的呢？

    if df.shape[0]>0:
        for i, row in df.iterrows():
#             scode = row['TradingCode']
            scode = row['trading_code']
    #       合约交易代码：合约交易代码包含合约标的、合约类型、到期月份、行权价格等要素。
    #       上证50ETF期权合约的交易代码共有17位，具体组成为：第1至第6位为合约标的证券代码；
    #       第7位为C或P，分别表示认购期权或者认沽期权；
    #       第8、9位表示到期年份的后两位数字；第10、11位表示到期月份；
    #       第12位期初设为“M”，并根据合约调整次数按照“A”至“Z”依序变更，如变更为“A”表示期权合约发生首次调整，
    #       变更为“B”表示期权合约发生第二次调整，依此类推；第13至17位表示行权价格，单位为0.001元
            df.loc[i,'Modify'] = scode[11]
            #下面是给call 和 put 打标签，方便后面筛选
            if scode[6]=='C':
                df.loc[i,'CP']='c'
            elif scode[6]=='P':
                df.loc[i,'CP']='p'
                        
#         contract = df[['ContractCode','StrikePrice','ExerciseDate','CP','Modify']]
        contract = df[['code','exercise_price','exercise_date','CP','Modify']]
        return contract.values
    else:
        print(date,'这一天取不到期权合约！！')
        return np.NaN

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [7]:
start = time.perf_counter()
day_num = len(trade_days)
contract_list = []
for t, date in enumerate(trade_days):
    contract_list.append((date, get_daily_contracts(date)))
    
    ###############进度条#################
    c = (t / day_num) * 100
    k = round(c)
    a = "*" * k
    b = "." * (100 - k)
    dur = time.perf_counter() - start
    print("\r{:^3.0f}%[{}->{}]{:.2f}s".format(c,a,b,dur),end = "")

100%[****************************************************************************************************->]697.94s

In [8]:
import pickle
contract_name = '50ETF'
file_name = 'option_contracts_{}_({}to{}).pkl'.format(contract_name,
                                                      start_date,
                                                      end_date)
folder_name = 'contracts_list'
path = folder_name + '/' + file_name
with open (path,'wb') as f:
    pickle.dump(contract_list,f)